In [1]:
#Importing the libraries
import pandas as pd
import json

In [27]:
#Loading the dataframes from the given json data
df1 = pd.read_json('users.json', lines=True, orient='records')
df2=pd.read_json('receipts.json', lines=True, orient='records')
df3 = pd.read_json('brands.json', lines=True, orient='records')

In [246]:
#Considering the dataframe for users and extracting the required details
#Extracting the user_id and the createdDate fom this dataframe which is going to be a part of the data warehouse
Users=pd.DataFrame()
Users['User_ID']=df1['_id']
Users['Created Date']=df1['createdDate']
#Next, taking the Id and the created date values which are as a dictionary
Users['User_ID']=[d.get('$oid') for d in df1['_id']]
Users['Created Date']=[d.get('$date') for d in Users['Created Date']]
Users.head()

,User_ID,Created Date
0,5ff1e194b6a9d73a3a9f1052,1609687444800
1,5ff1e194b6a9d73a3a9f1052,1609687444800
2,5ff1e194b6a9d73a3a9f1052,1609687444800
3,5ff1e1eacfcf6c399c274ae6,1609687530554
4,5ff1e194b6a9d73a3a9f1052,1609687444800


In [224]:
#Checking for NA values
Users.isna().sum()

User_ID         0
Created Date    0
dtype: int64

In [259]:
#The user_id is a primary key and should not have duplicate values, so they were dropped
Users.drop_duplicates(inplace=True)

In [261]:
Users = Users.reset_index(drop=True)

In [29]:
#Considering the Brands table
#Extracting the brand-id,brand-name and bar-code which are required for analysis
Brands=pd.DataFrame()
Brands['Brand_ID']=df3['_id']
#The id is part of the dictionary, so following the same procedure as before
Brands['Brand_ID']=[d.get('$oid') for d in Brands['Brand_ID']]
Brands['Name']=df3['name']
Brands['barcode']=df3['barcode']
Brands.head()

,Brand_ID,Name,barcode
0,601ac115be37ce2ead437551,test brand @1612366101024,511111019862
1,601c5460be37ce2ead43755f,Starbucks,511111519928
2,601ac142be37ce2ead43755d,test brand @1612366146176,511111819905
3,601ac142be37ce2ead43755a,test brand @1612366146051,511111519874
4,601ac142be37ce2ead43755e,test brand @1612366146827,511111319917


In [225]:
#Checking for NA values in Brands Table 
Brands.isna().sum()

Brand_ID    0
Name        0
barcode     0
dtype: int64

In [263]:
#Checking for dupliacates in Brands
Brands.Brand_ID.duplicated().sum()

0

In [247]:
#Considering the receipts table
#From this 2 tables need to created, the User_receipts table and the rewards_receipts table
#First creating the User_receipts table
User_Receipts=pd.DataFrame()
User_Receipts['Receipt_ID']=df2['_id']
User_Receipts['Receipt_ID']=[d.get('$oid') for d in User_Receipts['Receipt_ID']]
User_Receipts['User_ID']=df2['userId']
User_Receipts['Total Spent']=df2['totalSpent']
User_Receipts['Items Count']=df2['purchasedItemCount']
User_Receipts['Reward Status']=df2['rewardsReceiptStatus']
User_Receipts['Date Scanned']=df2['dateScanned']
User_Receipts['Date Scanned']=[d.get('$date') for d in User_Receipts['Date Scanned']]
User_Receipts.head()

,Receipt_ID,User_ID,Total Spent,Items Count,Reward Status,Date Scanned
0,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6,26.0,5.0,FINISHED,1609687531000
1,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052,11.0,2.0,FINISHED,1609687483000
2,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b,10.0,1.0,REJECTED,1609687537000
3,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6,28.0,4.0,FINISHED,1609687534000
4,5ff1e1d20a7214ada1000561,5ff1e194b6a9d73a3a9f1052,1.0,2.0,FINISHED,1609687506000


In [251]:
#The date scanned and the created date in the first 2 tables is in microsecond format
#Converting it to the day,time,year format along with the timewould make for a clearer understanding of data
#The last 3 digits make up the microsecond format, so with that as assumption, the logic was made
def date_converter(X):
    X=X/1000
    X = pd.to_datetime(X, unit='s')
    X = X.astype('datetime64[s]')
    return X
User_Receipts['Date Scanned']=date_converter(User_Receipts['Date Scanned'])
Users['Created Date']=date_converter(Users['Created Date'])

In [265]:
Users.head()

,User_ID,Created Date
0,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04
1,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30
2,5ff1e1e8cfcf6c399c274ad9,2021-01-03 15:25:28
3,5ff1e1b7cfcf6c399c274a5a,2021-01-03 15:24:39
4,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37


In [264]:
#Checking for duplicates in user_receipts table
User_Receipts.Receipt_ID.duplicated().sum()

0

In [294]:
#To create the rewards_receipt table, receipt id and barcode needs to be extracted
#The barcode is part of a list of dictionaries in the rewardsReceiptItemList column
df2.head()


,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [320]:
#Transferring The rows with NA values to a different dataframe which can be used to review data 
#setting the condition for NA values
a=df2.isna().any(axis=1)
receipts_na=df2[a==True]
recepts_na = receipts_na.reset_index(drop=True)
receipts_na.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
12,{'$oid': '5ff1e1b60a7214ada100055c'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687478000},{'$date': 1609687478000},NaN,{'$date': 1609687478000},NaN,8850.0,{'$date': 1612365878000},10.0,"[{'barcode': '034100573065', 'description': 'M...",FLAGGED,290.0,5ff1e194b6a9d73a3a9f1052
13,{'$oid': '5f9c74f70a7214ad07000037'},750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1604089079000},{'$date': 1604089079000},NaN,{'$date': 1609687494000},{'$date': 1604089080000},750.0,{'$date': 1604002679000},11.0,"[{'barcode': '075925306254', 'competitiveProdu...",REJECTED,1.0,5f9c74f7c88c1415cbddb839
15,{'$oid': '5ff1e1e90a7214ada1000569'},NaN,NaN,{'$date': 1609687529000},{'$date': 1609687529000},NaN,{'$date': 1609687529000},NaN,NaN,NaN,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff1e1e9b6a9d73a3a9f10f6
17,{'$oid': '5ff1e1b40a7214ada100055b'},750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1609687476000},{'$date': 1609687476000},NaN,{'$date': 1609687477000},{'$date': 1609687476000},750.0,{'$date': 1609601076000},1.0,"[{'barcode': '075925306254', 'competitiveProdu...",FLAGGED,1.0,5ff1e1b4cfcf6c399c274a54


In [162]:
#Removing the NA values 
X=pd.DataFrame()
X=df2.dropna()
X = X.reset_index(drop=True)
X['_id']=[d.get('$oid') for d in X['_id']]
X.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
3,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
4,5ff1e1e40a7214ada1000566,750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1609687524000},{'$date': 1609687524000},{'$date': 1609687525000},{'$date': 1609687530000},{'$date': 1609687525000},750.0,{'$date': 1609601124000},1.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,3.25,5ff1e1e4cfcf6c399c274ac3


In [298]:
#Extracted the receipt numbers for rewards_receipt table based on the nuber of items purchased for that receipt number
Y=[]
X['purchasedItemCount']=X['purchasedItemCount'].astype(int)
for i in range(0,len(X)):
    for j in range(0,X.loc[i][10]):
        Y.append(X.loc[i][0])

In [309]:
len(Y)

7915

In [212]:
a=pd.DataFrame()
a['rewards']=X['rewardsReceiptItemList']


rewards    [{'brandCode': 'MISSION', 'competitorRewardsGr...
Name: 5, dtype: object

In [217]:
#s=[{'a':1,'b':2},{'a':3}]
#for i in s:
#    print(i.get('b'))
#logic used to get the values based on dictionary key    

In [307]:
#Extracting the barcodes based on the number of times the item of that brandcode was purchased
#Some of the quantity purchased has 'none' values, for these the barcode is populated as 'none'
n=[]
for i in a['rewards']:
    for j in i:
        try:
            for k in range(0,j.get('quantityPurchased')):
                n.append(j.get('barcode'))
        except:
            n.append(None)
len(n)    

7931

In [ ]:
#There are some issues with the quality of data for the receipt table from 'receipts.json'
#The quantity purchased mentioned in the receipts table does not coinicde with the quantity mentioned in the dictionary for the item list
#Hence,there is a discrepency in the number of elements for the barcodes and the receipt numbers after extraction